In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add parent directory to sys.path in order to find common module
import sys
sys.path.insert(0,'../')

import common.datautils as datautils

In [ ]:
data = pd.read_csv('flag_effects.20180828-232133.csv')
data.loc[:, 'Energy'] *= 1e-6 # Convert energy to Joules
flag_effect_data = data.groupby(['Benchmark','DisabledFlag', 'Flags', 'Success'], as_index=False).agg({'Energy':'mean', 'Time':'mean'})
flag_effect_data.loc[:, 'Benchmark'] = flag_effect_data['Benchmark'].str.lower()
flag_effect_data = flag_effect_data[flag_effect_data["DisabledFlag"] != '-O3']
flag_effect_data.head()

In [ ]:
benchmarks = flag_effect_data["Benchmark"].unique()
print(benchmarks)

In [ ]:
ce_data = datautils.load_ce_results('CE.results.zip')
best_known = []

for benchmark in benchmarks:
    best_known.append(datautils.best_configuration_data('Energy', benchmark, ce_data, ['Benchmark', 'Flags', 'Energy', 'Time']))
    
best_known = pd.DataFrame(best_known)
best_known.loc[:, 'Benchmark'] = best_known['Benchmark'].str.lower()
best_known.set_index('Benchmark', drop=True, inplace=True)    

In [ ]:
for benchmark in benchmarks:
    print(benchmark)
    
    benchmark_data = flag_effect_data[flag_effect_data["Benchmark"] == benchmark]
    
    best_known_energy = best_known.loc[benchmark]['Energy']
    best_known_time = best_known.loc[benchmark]['Time']
    
    # Remember! Flags that have a positive effect when removed are actually negative
    negative_effect_time = benchmark_data[benchmark_data["Time"] / best_known_time < 0.95] 
    print("Negative Effect on Time:")
    print(negative_effect_time["DisabledFlag"].values)
    print("")
    
    negative_effect_energy = benchmark_data[benchmark_data["Energy"] / best_known_energy < 0.95]
    print("Negative Effect on Energy:")
    print(negative_effect_energy["DisabledFlag"].values)
    
    print("------\n")


In [ ]:
for benchmark in benchmarks:
    print(benchmark)
    
    benchmark_data = flag_effect_data[flag_effect_data["Benchmark"] == benchmark]
    
    best_known_energy = best_known.loc[benchmark]['Energy']
    best_known_time = best_known.loc[benchmark]['Time']
    
    improvement_factor = 0.05
    
    # Remember! Flags that have a positive effect when removed are actually negative
    positive_effect_time = benchmark_data[benchmark_data["Time"] / best_known_time > (1 + improvement_factor)]
    print("Positive Effect on Time:")
    print(positive_effect_time["DisabledFlag"].values)
    print("")
    
    # Remember! Flags that have a positive effect when removed are actually negative
    positive_effect_energy = benchmark_data[benchmark_data["Energy"] / best_known_energy > (1 + improvement_factor)]
    print("Positive Effect on Energy:")
    print(positive_effect_energy["DisabledFlag"].values)
    
    print("------\n")
